## Compte rendu R5.A.10, ANDRIAMISA Nayann, Groupe Nayru

#### Brouillon
- Redis
- MongoDB

In [4]:
pip install redis

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pymongo

  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/c8/1a/1ba5578cc3acc8ef0dfd6eb9385f9c4b0fe3039665af93bcf062c34fcdb6/pymongo-4.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
    --------------------------------------- 10.2/472.7 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/472.7 kB 1.7 MB/s eta 0:00:01
   ------------------------- -------------- 307.2/472.7 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 472.7/472.7 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/300.4 kB ? eta -:--:--
   ---------------------------------------  297.0/300.4 kB 9.2 MB/s eta 0:00:01
   --------------------------------------